In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
from bs4 import BeautifulSoup
import tabula
import math

## 1. Funções

In [2]:
def download_file(url,path):
    '''
    This function download the pdfs files from urls.
    
    Parameters:
        url (string): the url file to download.
        path (string): the path to save the pdf file after download.
        
    Return:
        Local pdf file path.
    '''
    local_filename = url.split('/')[-1]
    local_filename = path+'/'+local_filename 
    
    if(local_filename[-4:] != '.pdf'):
        local_filename = local_filename + '.pdf'
        
    r = requests.get(url)
    f = open(local_filename, 'wb')
    
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk: 
            f.write(chunk)
    f.close()
    return local_filename

In [3]:
def create_folder(path):
    '''
    Function to create folder to save the files.
    
    Parameter:
        - path (string): absolute path file name.
    Return:
        - status of creation file.
    '''

    try:
        if(os.path.isdir(path)):
            print ("A pasta %s já existe" % path)
            return True
        os.mkdir(path)
    except OSError:
        print ("Criação da pasta %s falhou" % path)
        return False
    else:
        print ("A pasta %s foi criada com sucesso" % path)
        
    return True



In [4]:
def get_pdf_filename(dir='\\resultados'):
    '''
    Fuction to get absolute file name.
    
    Parameter:
        - dir (string): initial directory to reveal all the absolute filename.
    Return:
        - list(str): list of all the absolute filename.
    '''
    path = os.getcwd()+dir

    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.pdf' in file:
                files.append(os.path.join(r, file))

    return files

In [165]:
def read_page(pdf_filename):
    '''
    Fuction to get the borders limit and set the area selection on function read_pdf. 
    The parameter multiple_tables in tabula.read_pdf function was setted True because the tables in documents have different
    number of columns. So, it is to avoid this error.
    
    Parameter:
        - pdf_filename (string): pdf filename path.
    Return:
        - list(DataFrame): list of dataframes of each page. 
    '''
    left = 0
    top = 0
    bottom = 770
    right = 550
    test_area = [top, left, bottom, right]
    #print(test_area)
    return tabula.read_pdf(pdf_filename,stream=False,guess=True,multiple_tables=True,pages='all',area=test_area,silent=True,pandas_options={'header': None}) 

In [153]:
def get_medal(idx,nota,medal_index):
    '''
    Fuction to get the borders limit and set the area selection on function read_pdf. 
    The parameter multiple_tables in tabula.read_pdf function was setted True because the tables in documents have different
    number of columns. So, it is to avoid this error.
    
    Parameter:
        - idx (int): row index.
        - nota (float): 
        - medal_index (list(int)): 
    Return:
        - string: medal name. 
    '''
    if(medal_index[0] < idx and idx < medal_index[1]):
        return 'Ouro'
    elif(medal_index[1] < idx and idx < medal_index[2]):
        return 'Prata'
    elif(medal_index[2] < idx and idx < medal_index[3]):
        return 'Bronze'
    elif(medal_index[3] < idx and not math.isnan(nota)):
        return 'Honra'
    else:
        return 'Nenhuma'

In [6]:
results = 'resultados'
years = range(2016,2020)

if(create_folder(results)):
    for year in years:
        create_folder(results+'/'+str(year))

A pasta resultados já existe
A pasta resultados/2016 já existe
A pasta resultados/2017 já existe
A pasta resultados/2018 já existe
A pasta resultados/2019 já existe


## 2. Coletando e armazenando os dados da Olimpíada Nacional de Ciências (ONC)

### 2.1 ONC 2019

- Os dados da ONC 2019 estão disponíveis em uma página html. O bloco de código a seguir acessa a página, identifica onde as informações de interesse se encontram no código html e extrai essas informações.

In [8]:
url = "https://onciencias.org/resultado/resultado-final-onc-2019/"
url_home = 'https://onciencias.org/'

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []
# Encontro todos as tags 'a' e seleciono apenas as que possuem a substring 'medalha' ou 'mencao-honrosa'
for link in soup.find_all('a'):
    temp_link = link.get('href')
    if(str(temp_link).find('medalha') != -1 or str(temp_link).find('mencao-honrosa') != -1):
        links.append(url_home+temp_link[1:])
        #print(url_home+temp_link[1:])

# Extrair dos links as informações da série e da premiação para serem incluídas no dataframe
serie = []
medalhas = []
for link in links:
    temp_splited_link = link.split('/')
    serie.append(temp_splited_link[-2])
    medalhas.append(temp_splited_link[-1])


- Inserindo as informações no DataFrame

In [9]:
# Leitura dos dados da página e inclusão das informações no dataframe
for a,b,c in zip(serie,medalhas,links):
    df = pd.read_html(c)[0]
    df['serie'] = a
    df['medalha'] = b
    dfs.append(df)
appended_data = pd.concat(dfs)

In [10]:
appended_data.head()

,Nome do Aluno,Escola,Estado,Cidade,Unnamed: 4,serie,medalha
0,LARA DANTAS DE OLIVEIRA MOISES,7 DE SETEMBRO COLEGIO - NGS,CEARÁ,Fortaleza,NaN,9-ano-ensino-fundamental,medalha-ouro
1,NAILTON GAMA DE CASTRO A,MASTER COLEGIO,CEARÁ,Fortaleza,NaN,9-ano-ensino-fundamental,medalha-ouro
2,RAFAEL MORENO RIBEIRO,COLEGIO MILITAR DE SALVADOR,BAHIA,Salvador,NaN,9-ano-ensino-fundamental,medalha-ouro
3,ARTHUR PINTO LORENZO,COLEGIO MILITAR DE SALVADOR,BAHIA,Salvador,NaN,9-ano-ensino-fundamental,medalha-ouro
4,GUILHERME Z. HAUSSEN,COLEGIO ANCHIETA,RIO GRANDE DO SUL,Porto Alegre,NaN,9-ano-ensino-fundamental,medalha-ouro


In [42]:
# Separação em diferentes dataframes de acordo com a série do participante para salvar os dados em arquivos diferentes
series = list(appended_data['serie'].unique())
for serie in series:
    temp_df = appended_data[appended_data['serie'] == serie]
    temp_df.to_csv('resultados/2019/'+serie.replace('-','_')+'.csv')

### 2.2 ONC 2018

- Os arquivos da ONC 2018 estão disponíveis na internet em arquivos PDF seperados de acordo com a série dos estudantes. Então, será realizado o download desses arquivos para posteriormente serem processados e extrair as informações dos arquivos.

In [11]:
# lista dos links para o download dos arquivos pdf
links_onc_2018= ['http://onciencias.org/resultados/download/15',
                'http://onciencias.org/resultados/download/16',
                'http://onciencias.org/resultados/download/17',
                'http://onciencias.org/resultados/download/18']

In [35]:
# Acessando as páginas de download dos arquivos do resultados da olimpíada e salvando os arquivos pdf
for i in links_onc_2018:
    download_file(i,'resultados/2018')

### 2.3 ONC 2016

- Os arquivos da ONC 2016 estão disponíveis na internet em arquivos PDF seperados de acordo com a série dos estudantes. Então, será realizado o download desses arquivos para posteriormente serem processados e extrair as informações dos arquivos.

In [12]:
# lista dos links para o download dos arquivos pdf
links_onc_2016 = ['https://onciencias.org/resultados/download/9/ResultadoONC2016-Nivel%20D.pdf',
'https://onciencias.org/resultados/download/8/ResultadoONC2016-Nivel%20C.pdf',
'https://onciencias.org/resultados/download/7/ResultadoONC2016-Nivel%20B.pdf',
'https://onciencias.org/resultados/download/6/ResultadoONC2016-Nivel%20A.pdf']

In [36]:
# Acessando as páginas de download dos arquivos do resultados da olimpíada e salvando os arquivos pdf
for i in links_onc_2016:
    download_file(i,'resultados/2016')

## 3. Processando os arquivos pdf

- Os arquivos que foram salvos agora serão processados para as informações serem extraídas e manipuladas.

### 3.1 Processando os arquivos pdf dos resultados de 2016

- Listagem dos arquivos de 2016

In [7]:
files_2016 = get_pdf_filename('\\resultados\\2016')
files_2016

['C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20A.pdf',
 'C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20B.pdf',
 'C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20C.pdf',
 'C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20D.pdf']

- Conversão dos arquivos pdf em csv

In [5]:
for f in files_2016:
    tabula.convert_into(f, f.replace('.pdf','.csv'), output_format="csv", pages='all')

- Conferindo os arquivos

In [6]:
pd.read_csv(files_2016[0].replace('.pdf','.csv'),encoding='latin-1').tail()

,Nome,Escola,Cidade,UF,Nota,Escore
1452,Letícia Furtado Silva,E E Pe Anchieta,Coqueiral,MG,NaN,NaN
1453,NaN,NaN,NaN,NaN,NaN,NaN
1454,Luis Felipe Barbosa Lopes,Colégio Bom Jesus Santo Antonio,Rolandia,PR,NaN,NaN
1455,NaN,NaN,NaN,NaN,NaN,NaN
1456,Lucas Morais Ferreira,Ceefmti Daniel Comboni,Ecoporanga,ES,NaN,NaN


> Após ler o arquivo pdf, foi identificado que a última página do primeiro arquivo não foi lida. Será utilizado uma abordagem que aumenta a área de seleção. [Link do tutorial](https://aegis4048.github.io/parse-pdf-files-while-retaining-structure-with-tabula-py).

- Foi desenvolvido a função `read_page` que recebe um arquivo pdf e converte em um dataframe.

In [9]:
filename = files_2016[3]
dfs = read_page(filename)

- Verificando arquivo

In [11]:
dfs[0].head(15)

,0,1,2,3,4,5
0,Nome,Escola,Cidade,UF,Nota,E
1,NaN,NaN,NaN,NaN,NaN,NaN
2,O U R O,NaN,NaN,NaN,NaN,NaN
3,Felipe Reyel Feitosa de Sousa,Instituto Dom Barreto,Teresina,PI,75,1
4,Diego Ferreira Caldas,Colégio Militar,Belo Horizonte,MG,74,9
5,Alícia Fortes Machado,Instituto Dom Barreto,Teresina,PI,"71,5",9
6,Antônio Anderson Costa Pereira,Ari de Sa Cavalcante,Fortaleza,CE,70,9
7,Victória Moreira Reis Cogo,Instituto Dom Barreto,Teresina,PI,70,9
8,NaN,NaN,NaN,NaN,NaN,NaN
9,P R A T A,NaN,NaN,NaN,NaN,NaN


In [13]:
dfs[-1].tail()

,0,1,2,3,4,5
27,Gabriel Pradela,Colégio Objetivo Arujá,Aruja,SP,NaN,NaN
28,Gabriel Kashiwazaki,Colégio Elite,Campinas,SP,NaN,NaN
29,Renata Cristiane Rodrigues Ferreira,Colégio Objetivo Mairiporã,Mairipora,SP,NaN,NaN
30,Fernanda Lopes Ramalho,Etec De Monte Mor,Monte Mor,SP,NaN,NaN
31,Matheus Fernandes Flores,Votuporanguense De Ensino Escola,Votuporanga,SP,NaN,NaN


> Existem linhas inteiras com valor `NaN` e linhas com apenas a colunas `Nome` com valor `NaN` e demais colunas com valores, estas linhas são as primeiras de cada página do documento que possuem um erro na marcação da célula e a função não conseguiu extrair a informação destas células. Foi testado algumas alternativas sem êxito, então em outro momento voltaremos para esta situação. 

Ações:
- Unir os DataFrame em apenas 1
- Remover linhas com todas as colunas com valores `NaN`
- Converter a coluna Nota para float, atualmente ela é uma string
- Identificar criar a coluna `medalha` e preenchê-la
- Remover linhas com o nome da premição (`Ouro`, `Prata`, `Bronze` e `Honrosa`)
- Calcular o Score

- Concatenando a lista de dataframes

In [14]:
df_uni = pd.concat(dfs)
df_uni.shape

(527, 6)

In [15]:
df_uni.head()

,0,1,2,3,4,5
0,Nome,Escola,Cidade,UF,Nota,E
1,NaN,NaN,NaN,NaN,NaN,NaN
2,O U R O,NaN,NaN,NaN,NaN,NaN
3,Felipe Reyel Feitosa de Sousa,Instituto Dom Barreto,Teresina,PI,75,1
4,Diego Ferreira Caldas,Colégio Militar,Belo Horizonte,MG,74,9


- Renomeando as colunas

In [24]:
cols = ['Nome', 'Escola', 'Cidade', 'UF', 'Nota','Escore']
df_uni.columns = cols

- Removendo as linhas com valores `NaN` e removendo a linha que possui os valores do header

In [70]:
df_uni = df_uni.dropna(axis=0,how='all')
df_uni = df_uni.drop(0)
df_uni.head()

,Nome,Escola,Cidade,UF,Nota,Escore,NotaF
2,O U R O,NaN,NaN,NaN,NaN,NaN,NaN
3,Felipe Reyel Feitosa de Sousa,Instituto Dom Barreto,Teresina,PI,75,1,75
4,Diego Ferreira Caldas,Colégio Militar,Belo Horizonte,MG,74,9,74
5,Alícia Fortes Machado,Instituto Dom Barreto,Teresina,PI,"71,5",9,71.5
6,Antônio Anderson Costa Pereira,Ari de Sa Cavalcante,Fortaleza,CE,70,9,70


- Convertendo a coluna `Nota` para `float`

In [73]:
df_uni['NotaF'] = pd.to_numeric(df_uni['Nota'].str.replace(',','.'), downcast='float')
df_uni = df_uni.drop(['Nota'], axis=1)
df_uni.head()

,Nome,Escola,Cidade,UF,Nota,Escore,NotaF
2,O U R O,NaN,NaN,NaN,NaN,NaN,NaN
3,Felipe Reyel Feitosa de Sousa,Instituto Dom Barreto,Teresina,PI,75,1,75.0
4,Diego Ferreira Caldas,Colégio Militar,Belo Horizonte,MG,74,9,74.0
5,Alícia Fortes Machado,Instituto Dom Barreto,Teresina,PI,"71,5",9,71.5
6,Antônio Anderson Costa Pereira,Ari de Sa Cavalcante,Fortaleza,CE,70,9,70.0


- Identificando os índices que possuem o nome da `medalha`

In [121]:
df_uni = df_uni.reset_index()

In [124]:
df_uni = df_uni.drop(['index'], axis=1)

In [125]:
medal_index = df_uni.index[df_uni['Nome'].isin(['O U R O','P R A T A','B R O N Z E','M E N Ç Ã O    H O N R O S A'])]
medal_index

Int64Index([0, 6, 15, 33], dtype='int64')

- Criando coluna `medalha` e atribuindo o nome da medalha

In [131]:
df_uni["medalha"] = ''

In [132]:
for index, row in df_uni.iterrows():
    df_uni.at[index, 'medalha'] = get_medal(index,row['NotaF'],medal_index)

- Removendo as linhas divisoras de medalhas

In [137]:
df_uni = df_uni.drop(medal_index)

In [138]:
df_uni.head()

,Nome,Escola,Cidade,UF,Nota,Escore,NotaF,medalha
1,Felipe Reyel Feitosa de Sousa,Instituto Dom Barreto,Teresina,PI,75,1,75.0,Ouro
2,Diego Ferreira Caldas,Colégio Militar,Belo Horizonte,MG,74,9,74.0,Ouro
3,Alícia Fortes Machado,Instituto Dom Barreto,Teresina,PI,"71,5",9,71.5,Ouro
4,Antônio Anderson Costa Pereira,Ari de Sa Cavalcante,Fortaleza,CE,70,9,70.0,Ouro
5,Victória Moreira Reis Cogo,Instituto Dom Barreto,Teresina,PI,70,9,70.0,Ouro


- Calculando o Escore

In [142]:
df_uni['Escore'] = (df_uni['NotaF']/max(df_uni['NotaF']))*100

- Criando a coluna `serie`

In [151]:
df_uni['serie'] = '3 Ano'

In [155]:
df_uni.to_csv('2016_3ano.csv')

### 3.2. Processar demais arquivos de 2016

In [202]:
def read_page(pdf_filename):
    '''
    Fuction to get the borders limit and set the area selection on function read_pdf. 
    The parameter multiple_tables in tabula.read_pdf function was setted True because the tables in documents have different
    number of columns. So, it is to avoid this error.
    
    Parameter:
        - pdf_filename (string): pdf filename path.
    Return:
        - list(DataFrame): list of dataframes of each page. 
    '''
    left = 0
    top = 0
    bottom = 770
    right = 600
    test_area = [top, left, bottom, right]
    #print(test_area)
    return tabula.read_pdf(pdf_filename,lattice=True,guess=True,multiple_tables=True,pages='all',area=test_area,silent=True,pandas_options={'header': None}) 

In [210]:
files_2016[0]

'C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20A.pdf'

In [208]:
filename = files_2016[0]
dfs = read_page(filename)

In [211]:
dfs[0].head(10)

,0,1,2
0,NaN,NaN,NaN
1,NaN,78,NaN
2,NaN,"77,5",NaN
3,NaN,76,NaN
4,NaN,75,NaN
5,NaN,75,NaN
6,NaN,75,NaN
7,NaN,75,NaN
8,NaN,75,NaN
9,NaN,75,NaN


In [182]:
dfs[2].dropna(axis=1,how='all')

,0,1,2,3,4,5
0,NaN,NaN,NaN,NaN,"63,84",NaN
1,Carlos Eduardo Barros Ferreira de Moraes,Colégio Motivo,Recife,PE,53,"63,84"
2,Thiago Louro Fernandes,Colégio Vital Brazil,São Paulo,SP,53,"63,84"
3,Roberto Mitsuaki Saito,Objetivo Integrado,São Paulo,SP,53,"63,84"
4,Pedro Henrique Monteiro Tavares,Ari de Sa Cavalcante,Fortaleza,CE,52,"62,63"
5,João Victor Rodrigues Menezes,Ari de Sa Cavalcante,Fortaleza,CE,52,"62,63"
6,Jeffté Cunha de Morais,Farias Brito,Fortaleza,CE,52,"62,63"
7,Sabrina Ellen de Souza Aquino,Farias Brito,Fortaleza,CE,52,"62,63"
8,Igor Pinheiro Henriques de Araújo,Colégio Militar,Recife,PE,52,"62,63"
9,Guilherme Lins de Araújo,Grupo Genese de Ensino,Recife,PE,52,"62,63"


In [161]:
df_uni = pd.concat(dfs)
df_uni.shape

(624, 6)

In [162]:
cols = ['Nome', 'Escola', 'Cidade', 'UF', 'Nota','Escore']
df_uni.columns = cols

In [163]:
df_uni = df_uni.dropna(axis=0,how='all')
df_uni = df_uni.drop(0)
df_uni.head()

,Nome,Escola,Cidade,UF,Nota,Escore
2,NaN,Nível C (2o. Ano),NaN,NaN,NaN,NaN
4,NaN,RESULTADO,NaN,NaN,NaN,NaN
7,Nome,Escola Cidade,UF,Nota,Escor,NaN
9,O U R O,NaN,NaN,NaN,NaN,NaN
10,André Barreto Moreira,Ari de Sa Cavalcante Fortaleza,CE,83,"100,0",NaN


In [164]:
df_uni['NotaF'] = pd.to_numeric(df_uni['Nota'].str.replace(',','.'), downcast='float')
df_uni = df_uni.drop(['Nota'], axis=1)
df_uni.head()

ValueError: Unable to parse string "Escor" at position 2

## TODO
- Remove empty rows
- Apply this process to all 2016 files

In [262]:
dfs['Nota'] = pd.to_numeric(dfs['Nota'], downcast="float")

In [254]:
not math.isnan(dfs.iloc[-1]['Nota'])

False

In [252]:
dfs.iloc[100]['Nota']

'41,5'

- Salvando o arquivo pdf em formato csv

In [123]:
df[0].to_csv(files_2016[0].replace('.pdf','.csv'),index=False)

- Removendo linhas com valores nulos

In [10]:
pd.read_csv(files_2016[0].replace('.pdf','.csv'),encoding='utf8').tail()

,Nome,Escola,Cidade,UF,Nota,Escore
1370,Bruna Rafaela Barbosa E Silva,Dr Eloy de Souza,Acu,RN,NaN,NaN
1371,NaN,NaN,NaN,NaN,NaN,NaN
1372,Tanise Vieira,Colégio Dom Feliciano,Gravatai,RS,NaN,NaN
1373,NaN,NaN,NaN,NaN,NaN,NaN
1374,Tcharly Miguel Santana Santos,Escola Viva,Carmopolis,SE,NaN,NaN


In [34]:
df = pd.read_csv(files_2016[0].replace('.pdf','.csv'),encoding='utf8')
df.head()

,Nome,Escola,Cidade,UF,Nota,Escore
0,O U R O,NaN,NaN,NaN,NaN,NaN
1,Pedro Jatobá Arteiro,Colégio Santa Maria,Recife,PE,78,"100,0"
2,Amanda Onofre Neves Nóbrega,Antares Colégio,Fortaleza,CE,"77,5","99,4"
3,Thiago Eklésio Silveira Peixoto,Farias Brito,Fortaleza,CE,76,"97,4"
4,Marco Antônio de Lima Sampaio,Colégio Contato,Maceio,AL,75,"96,2"


In [15]:
df.dropna(how='all',inplace=True)
df.tail()

,Nome,Escola,Cidade,UF,Nota,Escore
1366,Everton Daniel da Silva,Escola Aluisio Germano,Carpina,PE,NaN,NaN
1368,Alexandre da Silva Gomes,Dr Eloy de Souza,Acu,RN,NaN,NaN
1370,Bruna Rafaela Barbosa E Silva,Dr Eloy de Souza,Acu,RN,NaN,NaN
1372,Tanise Vieira,Colégio Dom Feliciano,Gravatai,RS,NaN,NaN
1374,Tcharly Miguel Santana Santos,Escola Viva,Carmopolis,SE,NaN,NaN


In [35]:
df.sample(10)

,Nome,Escola,Cidade,UF,Nota,Escore
932,NaN,NaN,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN,NaN,NaN
723,Iara Dorigon,Colégio Olimpo,Palmas,TO,NaN,NaN
695,Gabriel Rodrigues Carrijo,Col Exitus,Uberlandia,MG,NaN,NaN
536,Paulo Vitor Silva Santos,Colégio Cristo Rei,Maceio,AL,NaN,NaN
41,Joaquim Miguel Moreira Santiago,Ari de Sá Cavalcante,Fortaleza,CE,"67,5","86,54"
321,Ana Gabriela Ponte Farias,NaN,Antares Colégio,Fortaleza,CE,NaN
807,NaN,NaN,NaN,NaN,NaN,NaN
1241,Guilherme Gomes de Souza,EMEF Rodrigo de Argolo Caracas,Guaramiranga,CE,NaN,NaN
510,Ana Luiza Correia Veras,Antares Colégio,Fortaleza,CE,NaN,NaN


In [ ]:
125

In [61]:
def read_pdf(filename):
    tables = tabula.read_pdf(filename, output_format="json", pages=2, silent=True)
    top = tables[0]["top"]
    left = tables[0]["left"]
    bottom = tables[0]["height"] + top
    right = tables[0]["width"] + left

    test_area = [top, left, bottom, right]

    return tabula.read_pdf(filename,multiple_tables=True,pages="all",area=test_area,silent=True)

In [ ]:
df = read_pdf(files_2016[0])

In [67]:
df[2].shape

(50, 6)

In [70]:
df[3].head(20)

,Unnamed: 0,52,Unnamed: 1
0,NaN,52,NaN
1,NaN,52,NaN
2,NaN,"51,5",NaN
3,NaN,51,NaN
4,NaN,51,NaN
5,NaN,51,NaN
6,NaN,51,NaN
7,NaN,51,NaN
8,NaN,51,NaN
9,NaN,51,NaN


In [78]:
while True:
    try:
        df = tabula.read_pdf(files_2016[0],multiple_tables=True,pages=21,silent=True)
        break
    except Exception:
        print("Oops!  That was no valid number.  Try again...")
        break

Error from tabula-java:
Exception in thread "main" java.lang.IndexOutOfBoundsException: Page number does not exist
	at technology.tabula.ObjectExtractor.extractPage(ObjectExtractor.java:19)
	at technology.tabula.PageIterator.next(PageIterator.java:29)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:165)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:128)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:109)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:79)




Oops!  That was no valid number.  Try again...
